In [152]:
import cv2
import numpy as np
import time

def filtrar_color_amarillo(imagen):
    # Convertir la imagen a formato HSV (Hue, Saturation, Value)
    hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    
    # Definir el rango de colores amarillos en HSV
    lower_yellow = np.array([18, 100, 100])
    upper_yellow = np.array([32, 255, 255])
    
    # Aplicar el filtro para detectar solo el color amarillo
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Crear una imagen binaria utilizando la máscara
    resultado = cv2.bitwise_and(imagen, imagen, mask=mask)
    
    return mask, resultado

def remove_small_particles(binary_image, min_area):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a mask to store the filtered particles
    filtered_mask = np.zeros_like(binary_image)
    
    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)
        
        # If the area is larger than the minimum area threshold, keep the contour
        if area >= min_area:
            cv2.drawContours(filtered_mask, [contour], -1, 255, cv2.FILLED)
    
    return filtered_mask


def remove_small_particles2(binary_image, min_area):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create lists to store positions and areas of particles
    particle_positions = []
    particle_areas = []
    
    # Create a mask to store the filtered particles
    filtered_mask = np.zeros_like(binary_image)
    
    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)
        
        # If the area is larger than the minimum area threshold, keep the contour
        if area >= min_area:
            # Get the centroid of the contour
            M = cv2.moments(contour)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            # Append position and area to the lists
            particle_positions.append([cx, cy])
            particle_areas.append(area)
            
            # Draw the contour on the filtered mask
            cv2.drawContours(filtered_mask, [contour], -1, 255, cv2.FILLED)
    
    # Convert lists to NumPy arrays
    particle_positions = np.array(particle_positions)
    particle_areas = np.array(particle_areas)
    
    return filtered_mask, particle_positions, particle_areas

def expand_binary_zone(binary_image, iterations=1):
    # Definir el kernel para la operación de dilatación
    kernel = np.ones((3, 3), np.uint8)
    
    # Aplicar la operación de dilatación
    expanded_image = cv2.dilate(binary_image, kernel, iterations=iterations)
    
    return expanded_image



def shrink_binary_zone(binary_image, iterations=1):
    # Define the kernel for the erosion operation
    kernel = np.ones((3, 3), np.uint8)
    
    # Apply the erosion operation
    shrunk_image = cv2.erode(binary_image, kernel, iterations=iterations)
    
    return shrunk_image

def filtering_1(imagen, mode=0, dif=3):
    mask_amarillo, resultado_amarillo = filtrar_color_amarillo(imagen)

    # Set the minimum area threshold
    height, width = mask_amarillo.shape
    min_area_threshold = int(height*width*0.00005)

    # Remove small particles from the binary image
    filtered_image, areas, positions = remove_small_particles2(mask_amarillo, min_area_threshold)

    filtered_image = expand_binary_zone(filtered_image, dif)
    filtered_image = shrink_binary_zone(filtered_image, dif)

    bordes = cv2.Canny(filtered_image, 100, 200)

    # Find contours in the binary image
    contours, _ = cv2.findContours(filtered_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If there are contours
    if contours:
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding rectangle of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Calculate the center of the bounding rectangle
        center_x = x + w // 2
        center_y = y + h // 2

        # Crop the image to the bounding rectangle
        cropped_image = imagen[y:y+h, x:x+w]

        # Display the cropped image if mode is not 0
        if mode != 0:
            cv2.imshow('Original', imagen)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            cv2.imshow('Cropped Image', cropped_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            cv2.imshow('Máscara Amarilla', mask_amarillo)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            cv2.imshow('Filtered Binary Image', filtered_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

    return bordes, filtered_image


In [153]:
for i in range(2, 3):  # Suponiendo que tienes imágenes de test1.jpg a test10.jpg
    # Formar el nombre de archivo
    filename = f'test{i}.jpg'

    # Cargar la imagen
    image = cv2.imread(filename)

    # Comprobar si la imagen se cargó correctamente
    if image is not None:
        filtering_1(image, mode=1)
    else:
        print(f'No se pudo cargar la imagen {filename}')

PRUEBAS

In [3]:
import cv2
import numpy as np

# Cargar la imagen
image = cv2.imread('test1.jpg')
cv2.imshow('Imagen original', image)
cv2.waitKey(0)

# Convertir la imagen a espacio de color HSV
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
cv2.imshow('Imagen HSV', hsv_image)
cv2.waitKey(0)

# Definir el rango de colores de la señal de tráfico en HSV
lower_yellow = np.array([18, 100, 100])
upper_yellow = np.array([32, 255, 255])

# Crear una máscara para los colores dentro del rango especificado
mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
cv2.imshow('Máscara', mask)
cv2.waitKey(0)

# Aplicar operaciones de morfología para mejorar la detección y rellenar el contorno de amarillo
kernel = np.ones((5, 5), np.uint8)
# Cierre para rellenar pequeños huecos dentro del contorno amarillo
mask_closed = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
# Apertura para eliminar detalles no deseados fuera del contorno amarillo
mask_cleaned = cv2.morphologyEx(mask_closed, cv2.MORPH_OPEN, kernel)

# Encuentra contornos en la máscara limpia
contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Crea una máscara en blanco del mismo tamaño que la imagen original
filled_contour_mask = np.zeros_like(mask_cleaned)

# Dibuja los contornos en la máscara en blanco (rellenándolos)
cv2.drawContours(filled_contour_mask, contours, -1, (255), thickness=cv2.FILLED)

# Muestra la máscara después de aplicar la morfología y rellenar los contornos amarillos
cv2.imshow('Máscara después de morfología y relleno', filled_contour_mask)
cv2.waitKey(0)

# Encontrar contornos en la máscara
contours, _ = cv2.findContours(filled_contour_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Encontrar el contorno más grande
largest_contour = max(contours, key=cv2.contourArea)

# Dibujar un rectángulo alrededor del contorno más grande
x, y, w, h = cv2.boundingRect(largest_contour)
cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Mostrar la imagen con las señales de tráfico resaltadas
cv2.imshow('Señal de tráfico resaltada', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, f1_score

# Aquí deberías tener tus etiquetas reales y predicciones
# labels = etiquetas reales
# predictions = predicciones del modelo

# Porcentaje de error
error_rate = np.mean(labels != predictions) * 100
print("Porcentaje de error: {:.2f}%".format(error_rate))

# Matriz de confusión
conf_matrix = confusion_matrix(labels, predictions)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matriz de Confusión')
plt.colorbar()
plt.xlabel('Etiqueta Predicha')
plt.ylabel('Etiqueta Real')
plt.show()

# Curva ROC
fpr, tpr, thresholds = roc_curve(labels, predictions)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

# Curva de precisión-recuperación
precision, recall, _ = precision_recall_curve(labels, predictions)
f1 = f1_score(labels, predictions)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', lw=2, label='Curva Precisión-Recuperación')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precisión-Recuperación')
plt.legend(loc="lower left")
plt.show()


ModuleNotFoundError: No module named 'sklearn'

In [133]:
import cv2
import numpy as np

# Cargar la imagen base y convertirla a escala de grises
base_image = cv2.imread('Rombo.jpg', cv2.IMREAD_GRAYSCALE)

# Binarizar la imagen base (opcional, dependiendo del preprocesamiento necesario)
_, base_thresholded = cv2.threshold(base_image, 127, 255, cv2.THRESH_BINARY)

# Encontrar contornos en la imagen base
base_contours, _ = cv2.findContours(base_thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Supongamos que solo hay un contorno en la imagen base
base_contour = base_contours[0]

# Cargar la imagen donde deseas detectar las señales de tráfico
input_image = cv2.imread('test3.jpg')

# Convertir la imagen de entrada a escala de grises
input_gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

# Binarizar la imagen de entrada (opcional, dependiendo del preprocesamiento necesario)
_, input_thresholded = cv2.threshold(input_gray, 127, 255, cv2.THRESH_BINARY)

# Detección de contornos en la imagen de entrada
contours, _ = cv2.findContours(input_thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterar sobre los contornos detectados
for contour in contours:
    # Calcula la similitud entre el contorno actual y el contorno de la imagen base
    similarity = cv2.matchShapes(contour, base_contour, cv2.CONTOURS_MATCH_I1, 0.0)
    
    # Define un umbral de similitud para filtrar los contornos
    threshold_similarity = 0.1
    
    # Si la similitud es mayor que el umbral, dibuja el contorno en la imagen de entrada
    if similarity < threshold_similarity:
        cv2.drawContours(input_image, [contour], -1, (0, 255, 0), 2)

# Muestra la imagen de entrada con los contornos de las señales de tráfico detectadas
cv2.imshow('Señales de tráfico detectadas', input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
try:
    from skimage import measure
except ModuleNotFoundError:
    print("Skimage library not found. Please install it using:")
    print("pip install scikit-image")
    exit()
# Cargar las imágenes
reference_image = cv2.imread('referencia.jpg')
input_image = cv2.imread('test7.jpg')

# Convertir las imágenes a espacio de color HSV
reference_hsv = cv2.cvtColor(reference_image, cv2.COLOR_BGR2HSV)
input_hsv = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV)

# Definir rangos de color para la señal de tráfico (ajustar según la señal)
lower_yellow = np.array([20, 100, 100], dtype="uint8")
upper_yellow = np.array([30, 255, 255], dtype="uint8")

# Crear máscaras para la señal de tráfico en ambas imágenes
reference_mask = cv2.inRange(reference_hsv, lower_yellow, upper_yellow)
input_mask = cv2.inRange(input_hsv, lower_yellow, upper_yellow)

# Aplicar técnicas de segmentación de imagen (opcional)
# Puedes aplicar técnicas como `morphological operations` o `watershed segmentation` para mejorar la segmentación de la señal de tráfico.

# Encontrar regiones de interés (regiones) en la imagen de referencia
reference_regions = measure.regionprops(reference_mask, properties=('area', 'major_axis_length', 'minor_axis_length'))

# Inicializar variables para la señal de tráfico en la imagen de entrada
best_similarity = 0.0
best_contour = None

# Iterar sobre las regiones de interés en la imagen de entrada
for region in measure.regionprops(input_mask, properties=('area', 'major_axis_length', 'minor_axis_length')):
    # Calcular la similitud entre la región actual y las regiones de la imagen de referencia
    for reference_region in reference_regions:
        similarity = calculate_similarity(region, reference_region)  # Implementar la función `calculate_similarity`
        if similarity > best_similarity:
            best_similarity = similarity
            best_contour = region.bbox

# Dibujar el contorno de la señal de tráfico detectada (si se encontró)
if best_contour is not None:
    x, y, w, h = best_contour
    cv2.rectangle(input_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Visualizar la imagen resultante
cv2.imshow('Señal de tráfico detectada', input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Skimage library not found. Please install it using:
pip install scikit-image


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 